In [ ]:
#!The one working

Mounted at /content/gdrive


In [5]:
# import tensorflow as tf
import numpy as np

In [ ]:
# !pip install pdfplumber
# !pip install nltk
# !pip install -U gensim
# !pip install flask-ngrok
# !pip install werkzeug
# !pip install numpy

In [6]:
import os

In [11]:
import os
import pdfplumber

def pdf_extract(file_name):
    directory = "C:/Users/MohdS/OneDrive/Desktop/Major-Project/Flask-QA-System/docs"
    pdf_txt = ""
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if(filename == file_name):
            pdf_txt = '' # Reset pdf_txt for each file
            with pdfplumber.open(os.path.join(directory, filename)) as pdf:  # Use os.path.join for path concatenation
                for pdf_page in pdf.pages:
                    single_page_text = pdf_page.extract_text()
                    pdf_txt = pdf_txt + single_page_text
    return pdf_txt


In [ ]:
extracted_text = pdf_extract("AMZN-Q3-2020-Earnings-Release.pdf")  # Replace "example.pdf" with the name of your PDF file
print(extracted_text)


In [9]:
print(pdf_extract('fdi.pdf'))

In [ ]:
#New Way pdf Extractor
#!pip install pdf_layout_scanner

In [ ]:
# from pdf_layout_scanner import layout_scanner
# toc=layout_scanner.get_toc('docs/test.pdf')
# pages=layout_scanner.get_pages('docs/test.pdf')
# print(len(pages))

# for i in range(len(pages)):
#     print(pages[i])

In [13]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [14]:
def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  import sklearn
  from sklearn.metrics.pairwise import cosine_similarity
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    if sim > max_sim:
      max_sim = sim
      index_sim = index
  
  return index_sim

===================================================================================================================

===================================================================================================================

In [21]:
# !pip install transformers==3.1.0
import torch
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

c:\Users\MohdS\OneDrive\Desktop\Major-Project\Flask\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
def bert_qa(question, context, max_len=500):

    #Tokenize input question and passage 
    #Add special tokens - [CLS] and [SEP]
    input_ids = tokenizer.encode (question, context,  max_length= max_len, truncation=True)  


    #Getting number of tokens in question and context passage that contains the answer
    sep_index = input_ids.index(102) 
    len_question = sep_index + 1   
    len_context = len(input_ids)- len_question  

    
    #Separate question and context 
    #Segment ids will be 0 for question and 1 for context
    segment_ids =  [0]*len_question + [1]*(len_context)  

    #Converting token ids to tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids) 


    #Getting start and end scores for answer
    #Converting input arrays to torch tensors before passing to the model
    start_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[0]
    end_token_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([segment_ids]) )[1]


    #Converting scores tensors to numpy arrays
    start_token_scores = start_token_scores.detach().numpy().flatten()
    end_token_scores = end_token_scores.detach().numpy().flatten()

    #Getting start and end index of answer based on highest scores
    answer_start_index = np.argmax(start_token_scores)
    answer_end_index = np.argmax(end_token_scores)


    #Getting scores for start and end token of the answer
    start_token_score = np.round(start_token_scores[answer_start_index], 2)
    end_token_score = np.round(end_token_scores[answer_end_index], 2)


    #Combining subwords starting with ## and get full words in output. 
    #It is because tokenizer breaks words which are not in its vocab.
    answer = tokens[answer_start_index] 
    for i in range(answer_start_index + 1, answer_end_index + 1):
        if tokens[i][0:2] == '##':  
            answer += tokens[i][2:] 
        else:
            answer += ' ' + tokens[i]  

    # If the answer not in the passage
    if ( answer_start_index == 0) or (start_token_score < 0 ) or  (answer == '[SEP]') or ( answer_end_index <  answer_start_index):
        answer = "Sorry, Couldn't find answer in given pdf. Please try again!"
    
    return (answer_start_index, answer_end_index, start_token_score, end_token_score,  answer)


In [17]:
def expand_split_sentences(pdf_txt):
  import nltk
  nltk.download('punkt')
  new_chunks = nltk.sent_tokenize(pdf_txt)
  length = len(new_chunks)
  #for i in range(length):
    #tmp_token = tokenizer.encode(new_chunks[i])
    #print('The input has a total of {:} tokens.'.format(len(tmp_token)))

  new_df = [];
  for i in range(length):
    paragraph = ""
    for j in range(i, length):
      #tmp_str = paragraph + new_chunks[j]
      tmp_token = tokenizer.encode(paragraph + new_chunks[j])
      length_token = len(tmp_token)
      if length_token < 510:
        #print(length_token)
        paragraph = paragraph + new_chunks[j]
      else:
        #print(length_token)
        break;
    #print(len(tokenizer.encode(paragraph)))
    new_df.append(paragraph)
  return new_df
  #for i in new_df:
    #print(i)

In [18]:
import torch

def bert_drive(file_name, question):
    import numpy
    text = pdf_extract(file_name)
    max_score = 0
    final_answer = ""
    new_df = expand_split_sentences(text)
    tokens = []
    s_scores = numpy.array([])
    e_scores = numpy.array([])

    for new_context in new_df:
        ans, score, start_score, end_score, token = bert_qa(question, new_context)
        if score > max_score:
            max_score = score
            # Convert scalar values to PyTorch tensors
            start_score_tensor = torch.tensor(start_score)
            end_score_tensor = torch.tensor(end_score)
            s_scores = start_score_tensor.detach().numpy()
            e_scores = end_score_tensor.detach().numpy()
            tokens = token
            final_answer = ans

    return final_answer, s_scores, e_scores, tokens

In [17]:
# question = "what is make in india?"
# answer = bert_drive('fdi.pdf', question)
# print(answer)
# # draw_graph(s_scores, e_scores, tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MohdS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


(444, array(0.85, dtype=float32), array(2.33, dtype=float32), 'initiative')


In [16]:
# print(answer)

('', array([], dtype=float64), array([], dtype=float64), [])


In [ ]:
# question = "What were the various steps taken by the government to boost domestic and foreign investment in India ?"

# answer = (bert_drive('fdi.pdf', question))
# print(answer)


In [1]:
from flask import *
from flask_ngrok import run_with_ngrok
from werkzeug.utils import secure_filename
from werkzeug.datastructures import  FileStorage

app = Flask(__name__)
run_with_ngrok(app)
@app.route('/', methods=['GET', 'POST'])
def index():
  if request.method == 'POST':
    if(request.form.get('btn') == 'index'):
      upload = request.files['upload']
      path = "docs"
      global file_name
      file_name = upload.filename
      print(file_name)

      import os
      if not os.path.exists('docs'):
        os.makedirs('docs')
      
      upload.save(os.path.join(path, secure_filename(upload.filename)))
      return redirect(url_for('qa'))
    elif (request.form.get('btn') == 'qa'):
      question = request.form.get('question')
      answer = ""
      # if(func == 'naive'):
      #   answer = naive_drive(file_name, question)
      # elif(func == 'w2v'):
      #   answer = word2vec_drive(file_name, question)
      # elif(func == 'glove'):
      #   answer = glove_drive(file_name, question)
      if(func == 'bert'):
        answer= bert_drive(file_name, question)
        # draw_graph(s_scores, e_scores, tokens)
      else:
        answer = "Currently Fine Tunning Not Supported"
        # answer = fine_tuning_drive(question)
      #answer = fine_tuning_drive(question)
      print(answer)
      return render_template('qa.html', answer = answer, question = question)
  return render_template('index.html')
@app.route('/upload/', methods=['GET', 'POST'])
def upload():
    global func
    func = request.args.get('type')
    return render_template('upload.html')

@app.route('/qa/', methods=['GET', 'POST'])
def qa():
    return render_template('qa.html')

In [ ]:
# !pip install pyngrok

In [ ]:
!ngrok authtoken ENTER_YOUR_AUTH_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
app.run()